In [222]:
import numpy as np
import pandas as pd           # for reading file
import pandas_profiling as pp # statistical visualise
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score
%matplotlib inline

In [223]:
train_df = pd.read_csv('../data/train.csv', index_col='Unnamed: 0', sep='\t')
test_df = pd.read_csv('../data/test.csv', index_col='Unnamed: 0', sep='\t')

In [224]:
# train_df.head(1)
# test_df.head(1)

In [225]:
print ("rows \t\t: ", train_df.shape[0])
print ("columns \t: ", train_df.shape[1])

rows 		:  30500
columns 	:  346


---
## 1. Опишите препроцессинг данных, инжиниринг фич и валидацию

#### Разбиваем выборку

In [226]:
X_train = train_df.drop(['0'], axis=1)
y_train = train_df['0']

X_validation = test_df.drop(['0'], axis=1)
y_validation = test_df['0']

In [227]:
print ("Train : ")
print ("\t", X_train.shape)
print ("\t", y_train.shape)
print ("Test : ")
print ("\t", X_validation.shape)
print ("\t", y_validation.shape)

Train : 
	 (30500, 345)
	 (30500,)
Test : 
	 (4166, 345)
	 (4166,)


#### Удаляем признаки у которых только 2 и меньше уникальных значения, и у которых вариабельность меньше 0.5%

In [228]:
def    clean_data(train, test):
    columns_name = []
    
    for i in test:
        if (len(train[i].unique()) == 1):
            columns_name.append(i)
        elif (len(train[i].unique()) == 2) and (len(test[i].unique()) <= 2):
            columns_name.append(i)
        elif (len(train[i].unique()) == 2) and min(train[i].value_counts()) / sum(train[i].value_counts()) * 100 < .5:
            columns_name.append(i)
    return columns_name

In [229]:
clean_columns = clean_data(X_train, X_validation)
X_train.drop(clean_columns, axis=1, inplace=True)
X_validation.drop(clean_columns, axis=1, inplace=True)

In [230]:
print ("Train : ")
print ("\t", X_train.shape)
print ("\t", y_train.shape)
print ("Test : ")
print ("\t", X_validation.shape)
print ("\t", y_validation.shape)

Train : 
	 (30500, 106)
	 (30500,)
Test : 
	 (4166, 106)
	 (4166,)


#### Удаляем признаки у которых уровень корреляция больше 99

In [199]:
# Удаляем признаки с корреляцией 1
def    check_corr_data(train, test, type_corr):
    corr_matrix = train.corr(method=type_corr).abs()
    
    upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))

    to_drop = []
    for column in upper.columns:
        if any(upper[column] > 0.99):
            train.drop(column, axis=1, inplace=True)
            test.drop(column, axis=1, inplace=True)
            print (column)
            return True
    return False

def     clean_corr_data(train, test, type_corr):
    while check_corr_data(train, test, type_corr):
        ;

In [231]:
clean_corr_data(X_train, X_validation, 'pearson')

329
333
334


In [232]:
print ("Train : ")
print ("\t", X_train.shape)
print ("\t", y_train.shape)
print ("Test : ")
print ("\t", X_validation.shape)
print ("\t", y_validation.shape)

Train : 
	 (30500, 103)
	 (30500,)
Test : 
	 (4166, 103)
	 (4166,)


#### Кодируем категориальные признаки one_hot_encoding

In [233]:
def one_hot_encoding(train, test):
    for i in train:
        if len(train[i].unique()) < 15 and len(train[i].unique()) > 1:
            train2 = pd.concat([train, pd.get_dummies(train[i], prefix=i)], axis=1)
            train.drop([i], axis=1, inplace=True)
            train = train2
            
            test2 = pd.concat([test, pd.get_dummies(train[i], prefix=i)], axis=1)
            test.drop([i], axis=1, inplace=True)
            test = test2
            
    return train, test

In [234]:
X_train, X_validation = one_hot_encoding(X_train, X_validation);

In [235]:
print ("Train : ")
print ("\t", X_train.shape)
print ("\t", y_train.shape)
print ("Test : ")
print ("\t", X_validation.shape)
print ("\t", y_validation.shape)

Train : 
	 (30500, 221)
	 (30500,)
Test : 
	 (30500, 221)
	 (4166,)


#### Опять чистим коррелирующие данные

In [236]:
clean_corr_data(X_train, X_validation, 'pearson')

120_0.0
127_0.0
212_1.0
312_0.0
312_0.39424
312_0.4324508
312_0.47097
312_0.5197096
312_0.5722851
312_0.588824
312_0.6608119
312_0.7575137
312_0.8421118000000001
312_0.9310675
312_1.0
313_0.0
313_0.016691400000000002
313_0.052069500000000005
313_0.0654514
313_0.1504556
313_0.2396119
313_0.277947
313_0.3472929
313_0.461724
313_0.6188458
313_1.0
314_0.0
314_0.1178385
314_0.1699219
314_0.202474
314_0.2057292
314_0.4466146
314_0.4889323
314_0.5833333
314_0.6126302
314_1.0
315_0.015435399999999997
315_0.033313800000000005
315_0.07876389999999997
315_0.3415823
315_0.4127147
315_0.4488797000000001
315_0.5901081
315_0.6529206
315_0.8525990999999999
315_0.9929259
315_1.0


In [237]:
print ("Train : ")
print ("\t", X_train.shape)
print ("\t", y_train.shape)
print ("Test : ")
print ("\t", X_validation.shape)
print ("\t", y_validation.shape)

Train : 
	 (30500, 174)
	 (30500,)
Test : 
	 (30500, 174)
	 (4166,)


---

In [210]:
# pp.ProfileReport(X_train)

In [17]:
# pp.ProfileReport(X_test)

---
## 2. Постройте логистическую регрессию. Укажите значение на валидации и публичном лидерборде

In [238]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

# X_train_1
# y_train_1

scaler = StandardScaler()
logit = LogisticRegression(random_state=17, class_weight='balanced')
logit_pipe_params = {'logit__C': np.logspace(-8, 8, 17)}


In [239]:
log_reg = GridSearchCV(logit, logit_pipe_params, scoring='roc_auc', n_jobs=-1, verbose=1)

log_reg.fit(X_train, y_train)
print (lr_g.best_params_)
print (lr_g.best_score_)

Fitting 3 folds for each of 17 candidates, totalling 51 fits


JoblibValueError: JoblibValueError
___________________________________________________________________________
Multiprocessing exception:
...........................................................................
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/runpy.py in _run_module_as_main(mod_name='ipykernel_launcher', alter_argv=1)
    188         sys.exit(msg)
    189     main_globals = sys.modules["__main__"].__dict__
    190     if alter_argv:
    191         sys.argv[0] = mod_spec.origin
    192     return _run_code(code, main_globals, None,
--> 193                      "__main__", mod_spec)
        mod_spec = ModuleSpec(name='ipykernel_launcher', loader=<_f...b/python3.6/site-packages/ipykernel_launcher.py')
    194 
    195 def run_module(mod_name, init_globals=None,
    196                run_name=None, alter_sys=False):
    197     """Execute a module's code without importing it

...........................................................................
/L

---
## 3. Постройте бэггинг на логистических регрессиях. Укажите значение на валидации и публичном лидерборде

---
## 4. Постройте случайный лес. Укажите значение на валидации и публичном лидерборде

---
## 5. Подберите лучший вариант простого ансамбля. Опишите его и укажите значение на валидации и публичном лидерборде

---
## 6. Сделайте стекинг. Опишите его и укажите значение на валидации и публичном лидерборде

---
## 7. Укажите ваш ник, значение на lb и валидации, опишите коротко решение

In [67]:
import numpy as np
import pandas as p
# Create feature matrix with two highly correlated features
X = np.array([[1, 1, 1],
              [2, 2, 0],
              [3, 3, 1],
              [4, 4, 0],
              [5, 5, 1],
              [6, 6, 0],
              [7, 7, 1],
              [8, 7, 0],
              [9, 7, 1]])

# Convert feature matrix into DataFrame
df = pd.DataFrame(X)

# View the data frame
df

,0,1,2
0,1,1,1
1,2,2,0
2,3,3,1
3,4,4,0
4,5,5,1
5,6,6,0
6,7,7,1
7,8,7,0
8,9,7,1


In [79]:
# Create correlation matrix
df[1]

corr_matrix = df.corr()

# Select upper triangle of correlation matrix
# upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))

# Find index of feature columns with correlation greater than 0.95
# to_drop = [column for column in upper.columns if any(upper[column] > 0.95)]

In [80]:
corr_matrix

,0,1,2
0,1.000000,0.976103,0.000000
1,0.976103,1.000000,-0.034503
2,0.000000,-0.034503,1.000000
